In [34]:
ax_path = "./data/lc_ax_male.csv"
bx_path = "./data/lc_bx_male.csv"
kt_path = "./data/lc_kt_male.csv"
path = [ax_path, bx_path, kt_path]

import Prob 
death_Prob = Prob.D_Prob()
death_Prob.read_abk(path)

import ST
import numpy as np
from statistics import mean
import numba

np.random.seed(8)  # 设置固定的随机数种子


计算一些死亡参数

In [35]:

import pandas as pd
import os

lifespan =92
x0 = 67
T0 = 2020
p = 1

terminal_live_cache_file = f'./data/terminal_live_cache_male_{x0}_{p}.csv'
interval_P_cache_file = f'./data/interval_P_cache_male_{x0}_{p}.csv'

# 尝试从CSV文件读取数据
if os.path.exists(terminal_live_cache_file):
    terminal_live_cache = pd.read_csv(terminal_live_cache_file, index_col=0).to_dict()['Value']
else:
    terminal_live_cache = {}

if os.path.exists(interval_P_cache_file):
    interval_P_cache_df = pd.read_csv(interval_P_cache_file)
    interval_P_cache = {(row['Start'], row['End']): row['Value'] for index, row in interval_P_cache_df.iterrows()}
else:
    interval_P_cache = {}

for i in range(0, (lifespan - x0) * p):
    if i == 0:
        terminal_live_cache[i] = death_Prob.accu_live(x0, T0, lifespan - x0, 0)
    else:
        terminal_live_cache[i] = death_Prob.interval_terminal_live(x0, T0, i, lifespan - x0, p)
    for j in range(i + 1, (lifespan - x0) * p + 1):
        if i == 0:
            interval_P_cache[(i, j)] = death_Prob.unit_P(x0, T0, j, p)
        else:
            interval_P_cache[(i, j)] = death_Prob.interval_death_P(x0, T0, i, j, p)

pd.DataFrame.from_dict(terminal_live_cache, orient='index', columns=['Value']).to_csv(terminal_live_cache_file)
interval_P_cache_df = pd.DataFrame([(key[0], key[1], value) for key, value in interval_P_cache.items()], columns=['Start', 'End', 'Value'])
interval_P_cache_df.to_csv(interval_P_cache_file, index=False)

In [36]:
interval_P_cache

{(0.0, 1.0): 0.010802011451874649,
 (0.0, 2.0): 0.01136908803373618,
 (0.0, 3.0): 0.01216763191243066,
 (0.0, 4.0): 0.012919424882604183,
 (0.0, 5.0): 0.013805025286351988,
 (0.0, 6.0): 0.014746785948507552,
 (0.0, 7.0): 0.015831392886872665,
 (0.0, 8.0): 0.0170574731446942,
 (0.0, 9.0): 0.018478569254075757,
 (0.0, 10.0): 0.020111235772431537,
 (0.0, 11.0): 0.021980714621955506,
 (0.0, 12.0): 0.024086210520932017,
 (0.0, 13.0): 0.026415569818527804,
 (0.0, 14.0): 0.028934829776936784,
 (0.0, 15.0): 0.031597315223410624,
 (0.0, 16.0): 0.03434553523485977,
 (0.0, 17.0): 0.0371186892573376,
 (0.0, 18.0): 0.0398539840447479,
 (0.0, 19.0): 0.042484366985194026,
 (0.0, 20.0): 0.04492924860345609,
 (0.0, 21.0): 0.04708215245987394,
 (0.0, 22.0): 0.04879942942479056,
 (0.0, 23.0): 0.04989767058599086,
 (0.0, 24.0): 0.05016680155872481,
 (0.0, 25.0): 0.04940321669954885,
 (1.0, 2.0): 0.011493238123566063,
 (1.0, 3.0): 0.012300502076727277,
 (1.0, 4.0): 0.013060504602891881,
 (1.0, 5.0): 0.0139

In [37]:
from numba import cuda, float32
d_terminal_live_cache = cuda.to_device(np.array(list(terminal_live_cache.values())))

np_interval_P_cache = np.zeros(((lifespan - x0) * p, (lifespan - x0) * p))
np_interval_P_cache[1][2]
for i,j in interval_P_cache.keys():
    np_interval_P_cache[int(i)][int(j-1)] = interval_P_cache[(i,j)]
d_interval_P_cache = cuda.to_device(np_interval_P_cache)

在GPU上计算单条轨道

In [38]:
from numba.cuda.random import create_xoroshiro128p_states, xoroshiro128p_uniform_float32, xoroshiro128p_normal_float32
import math

@cuda.jit
def single_mc(d_out1, d_out2, rng_states, d_i_P_c, d_t_l_c):
    # # 生成线程
    idx = cuda.grid(1)

    # # 解包参数
    # for key, value in kwargs:
    #     if key == "T":
    #         T = value
    #     elif key == "k":
    #         k = value
    #     elif key == 'S0':
    #         S0 = value
    #     elif key == "r":
    #         r = value
    #     elif key == "l":
    #         l = value
    #     elif key == "sigma":
    #         sigma = value
    #     elif key == "g":
    #         g = value
    
    T = 34
    S0 = 1
    r = 0.05
    g = 0.02
    k = 15
    l = 0.021555
    sigma = 0.2
    p = 1
    

    # 迭代轨道并且求和
    St = S0
    dt = float(1)/p #一年分成几份
    t = 0

    for w in range(T):
        
        # 轨道前进p步
        for _ in range(p):
            t += dt
            rand_x = xoroshiro128p_normal_float32(rng_states, idx)
            St = St * math.exp((r - l - 0.5 * sigma**2) * dt + sigma * math.sqrt(dt) * rand_x) # 考虑参数的转移

        final_int = math.exp(-r * t) * max(math.exp(g*t)-St, 0)
        
        # intern_res += int_item[int((w + 1) * buff_day) - 1] * interval_P_cache[(k, w+k+1)]
        d_out1[idx] += d_i_P_c[k][w+k] * final_int
        d_out2[idx] += math.exp(-l*t) * d_i_P_c[k][w+k]

        for _ in range(p):
            t += dt
            rand_x = xoroshiro128p_normal_float32(rng_states, idx)
            St = St*math.exp((r - l - 0.5 * sigma**2) * dt + sigma * rand_x) # 考虑参数的转移

        d_out1[idx] += St * d_t_l_c[k]
        d_out2[idx] = 1 - d_out2[idx] - d_t_l_c[k]*math.exp(-l*T/p)

    

    

In [39]:
M = 300000
d_res = cuda.to_device(np.zeros(M, dtype=np.float32))
d_fees = cuda.to_device(np.zeros(M, dtype=np.float32))
# 设定随即器
threads_per_block = 256
blocks = (M + threads_per_block - 1) // threads_per_block

rng_states = create_xoroshiro128p_states(threads_per_block * blocks, seed=1)

single_mc[blocks, threads_per_block](d_res, d_fees, rng_states, d_interval_P_cache, d_terminal_live_cache)

results = d_res.copy_to_host()
fees = d_fees.copy_to_host()

avg_res = np.mean(results)
fee_avg_res = np.mean(fees) 

In [40]:
print(avg_res, fee_avg_res)

40.939224 0.0009967461


并行计算蒙特卡洛

In [ ]:
def MC():
    

In [ ]:

def MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l, d_np_interval_P_cache, d_terminal_live_cache):
    T = (lifespan - x0) * p - k
    # print('T', T)
    N = buff_day * T
    # print('N', N)
    sigma = 0.2

    # 你现有的MC函数代码继续
    result = []
    avg_res = []
    ave = []
    fee_result = []

    M = 1000000
    d_results = cuda.to_device(np.zeros(M, dtype=np.float32))
    d_fee_results = cuda.to_device(np.zeros(M, dtype=np.float32))

    # 定义线程和块的数量
    threads_per_block = 256
    blocks_per_grid = (M + threads_per_block - 1) // threads_per_block

    t_all, int_item = ST.intg_item(N, p, T, S0, r, l, g, sigma)
    d_texp_all = cuda.to_device(np.exp(-l*t_all))
    d_ltp = cuda.to_device(np.exp(-l*T/p))
    print("generate done!")
    # 执行CUDA核函数
    single_mc[blocks_per_grid, threads_per_block](d_np_interval_P_cache, d_terminal_live_cache, d_texp_all, d_results, d_fee_results, buff_day, k, l, T, d_ltp, N, S0, r, g, sigma)

    # 将结果传回CPU
    results = d_results.copy_to_host()
    fees = d_fee_results.copy_to_host()
    avg_res = np.mean(results)
    fee_avg_res = np.mean(fees) 

    return avg_res, fee_avg_res, ave

In [ ]:
def preliminary_search_per_i(initial_l, k, step_size, M, p, buff_day, S0, r, g, x0, T0, lifespan, max_iter=100):
    T = lifespan - x0 - k
    # l = 0.000129 * T**2 - 0.010775 * T + 0.200646
    l = initial_l
    lower_l, upper_l = None, None  # 初始化 lower_l 和 upper_l

    # 首先减小 l 直到 fee_avg_res - avg_res[-1] 变为负值
    for _ in range(max_iter):
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l, d_np_interval_P_cache, d_terminal_live_cache)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        if fee_avg_res - avg_res < 0:
            upper_l = l + step_size  # 记录这个点的前一个点为 upper_l
            break
        l -= step_size

    # 从取负值的 l 开始，反转方向，以较小的步长增加 l
    step_size /= 3  # 减小步长
    for _ in range(max_iter):
        l += step_size
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l, d_np_interval_P_cache, d_terminal_live_cache)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        if fee_avg_res - avg_res > 0:
            lower_l = l - step_size  # 记录这个点的前一个点为 lower_l
            upper_l = l
            break

    print("Before lower_l, upper_l:", lower_l, upper_l)
    # 第二轮搜索：缩小搜索范围
    new_lower_l = lower_l  # 初始化新的搜索范围
    new_upper_l = upper_l

    l_mid = (lower_l + upper_l) / 2  # 找到当前范围的中点
    print("l_mid: ", l_mid)
    avg_res_mid, fee_avg_res_mid, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l_mid, d_np_interval_P_cache, d_terminal_live_cache)
    print("mid_max(exp(g*t), S_t): ", avg_res_mid)
    print("mid_fee_avg_res: ", fee_avg_res_mid)
    # 根据中点处的差值调整搜索范围
    if fee_avg_res_mid - avg_res_mid > 0:
        new_upper_l = l_mid
    else:
        new_lower_l = l_mid
        
    return new_lower_l, new_upper_l


In [ ]:
def fine_search_per_i(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, lower_l, upper_l, fine_step_size):
    best_l = lower_l + fine_step_size #lower_l时diff<0,所以从lower_l+fine_step_size查看diff和0的区别
    min_difference = np.inf
    found_small_diff = False  # 标记是否找到小于0.0003的差异
    
    l = lower_l + fine_step_size
    while l <= upper_l - fine_step_size:
        print("l: ", l)
        avg_res, fee_avg_res, _ = MC(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, l, d_np_interval_P_cache, d_terminal_live_cache)
        print("max(exp(g*t), S_t): ", avg_res)
        print("fee_avg_res: ", fee_avg_res)
        difference = abs(fee_avg_res - avg_res)  # 计算差值的绝对值
        # 如果找到了小于0.0003的差异，标记为True
        if difference <0.000003:
            break
    
        if difference < 0.0003:
            found_small_diff = True
        
        # 在找到小于0.0003的差异后，如果差异又大于0.0025，结束搜索
        if found_small_diff and difference > 0.002:
            print(f"Ending search: difference increased beyond 0.0025 after finding a smaller difference.")
            break  # 结束循环
        
        # 更新最小差异和最优l
        if difference < min_difference:
            min_difference = difference
            best_l = l
            
        l += fine_step_size  # 使用非常小的步长
    
    print(f"Best l before stopping: {best_l}, Min difference before stopping: {min_difference}")
    return best_l, min_difference



In [ ]:
# 初始化参数
M = 30000
x0 = 67
T0 = 2020
lifespan = 92


S0 = 1.0
p = 1
buff_p = int(12/p)
buff_day = 10*21*buff_p

r = 0.05
g = 0.02
tolerance = 1e-6 
step_size = 0.00006  #preliminary_search_per_i
k = 15   ####这次想要计算出来的年龄是多少的，k = 0,1,...,24
initial_l = 0.021555

best_ls = []  # 用于存储每个索引i对应的最优l值
min_differences = []  # 用于存储每个索引i对应的最小差值

print("k:", k)
lower_l, upper_l = preliminary_search_per_i(initial_l, k, step_size, M, p, buff_day, S0, r, g, x0, T0, lifespan, max_iter=100)
print(f"索引{k}, new lower_l, upper_l: {lower_l}, {upper_l}")

if lower_l is not None and upper_l is not None:
    fine_step_size = 0.000001 # 精细搜索步长 
    best_l, min_difference = fine_search_per_i(M, k, p, buff_day, S0, r, g, x0, T0, lifespan, lower_l, upper_l, fine_step_size)
    best_ls.append(best_l)
    min_differences.append(min_difference)
    print(f"Index {k}: Best l value: {best_l} with minimum difference: {min_difference}")
else:
    best_ls.append(None)
    min_differences.append(None)
    print(f"Index {k}: Unable to find suitable l interval during preliminary search.")

k: 15
l:  0.021555
generate done!


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
Unknown attribute 'intg_item' of type Module(<module 'ST' from '/home/hhf2/LPB-/option value/ST.py'>)

File "../../../../tmp/ipykernel_52707/1834391665.py", line 5:
<source missing, REPL/exec in use?>

During: typing of get attribute at /tmp/ipykernel_52707/1834391665.py (5)

File "../../../../tmp/ipykernel_52707/1834391665.py", line 5:
<source missing, REPL/exec in use?>
